In [1]:
import numpy as np
import pandas as pd
import spacy

In [2]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()


In [3]:
sent_df=pd.read_csv('../Downloads/archive (6)/train.csv',encoding='unicode_escape',usecols=['text','sentiment'])

In [4]:
from nltk.tokenize import word_tokenize
import re


In [5]:
sent_df.dropna(inplace=True)

In [6]:
sent_df

,text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD I will miss you here in San Diego!!!,negative
2,my boss is bullying me...,negative
3,what interview! leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...",negative
...,...,...
27476,wish we could come see u on Denver husband l...,negative
27477,I`ve wondered about rake to. The client has ...,negative
27478,Yay good for both of you. Enjoy the break - y...,positive
27479,But it was worth it ****.,positive


In [7]:
sent_df['sentiment']=le.fit_transform(sent_df['sentiment'])

In [8]:
sent_df

,text,sentiment
0,"I`d have responded, if I were going",1
1,Sooo SAD I will miss you here in San Diego!!!,0
2,my boss is bullying me...,0
3,what interview! leave me alone,0
4,"Sons of ****, why couldn`t they put them on t...",0
...,...,...
27476,wish we could come see u on Denver husband l...,0
27477,I`ve wondered about rake to. The client has ...,0
27478,Yay good for both of you. Enjoy the break - y...,2
27479,But it was worth it ****.,2


In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(sent_df['text'],sent_df['sentiment'],test_size=0.2)

In [14]:
X_train=X_train.apply(url_rem)
X_test=X_test.apply(url_rem)

In [15]:
import string
exclude=string.punctuation
def punc_rem(text):
    return text.translate(str.maketrans('','',exclude)).strip()

In [16]:
X_train=X_train.apply(punc_rem)
X_test=X_test.apply(punc_rem)

In [17]:
X_train=X_train.str.lower()
X_test=X_test.str.lower()

In [10]:
import re
exc=re.compile(r'\d+')
def number_adj(txt):
    return re.sub(exc,'',txt)

In [18]:
X_train=X_train.apply(number_adj)
X_test=X_test.apply(number_adj)

In [19]:
from textblob import TextBlob


In [31]:
def word_correction(text):
    txt=TextBlob(text).correct()
    return txt.string

(21984,)

In [37]:
X_train.sample(100)

16905    sanctuarysunday awesome news about the leo awa...
11642    i have myself barely  hrs of sleep and the lor...
12489                          omg i realy cant sleep ughh
860      is sitting thru the boring bits in titanic wai...
17335                                       crying so hard
                               ...                        
10015                 my twitter looks funny  someone help
670      aah well have a few friends over there just wo...
2064      years here  left dell okc for the last time a...
23071    dad just gave me his old blackberry not that o...
4874     my brain has become a converter ed ive been co...
Name: text, Length: 100, dtype: object

In [12]:
def url_rem(txt):
     url_pattern = re.compile(r'https?://\S+|www\.\S+')
     return url_pattern.sub("", txt)

In [27]:
from autocorrect import Speller
spell=Speller(lang="en")

In [28]:
def spell_correct(text):
    sptext = []
    for doc in text:
        sptext.append(' '.join([spell(w).lower() for w in (doc)]))      
    return sptext

In [30]:
spell_correct(['hellloo'])

['h e l l l o o']

In [42]:
from nltk.tokenize import word_tokenize
doc_list=[]

In [43]:
def tokwnizer(doc_):
    doc_list.append(word_tokenize(doc_))

In [48]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()


In [62]:
x_tr=cv.fit_transform(X_train)
x_te=cv.transform(X_test)

In [63]:
x_tr.shape

(21984, 23324)

In [64]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()


In [65]:
gnb.fit(x_tr.toarray(),y_train)

GaussianNB()

In [67]:
y_pred=gnb.predict(x_te.toarray())

In [53]:
y

GaussianNB()

In [69]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.4028384279475983

In [45]:
doc_list

[['you',
  'need',
  'to',
  'follow',
  'me',
  'so',
  'i',
  'could',
  'send',
  'you',
  'a',
  'direct',
  'message'],
 ['i',
  'was',
  'scared',
  'by',
  'the',
  'daleks',
  'in',
  'the',
  'dw',
  'exhibition',
  'in',
  'cardiff'],
 ['ok', 'tx', 'its', 'so', 'worth', 'the', 'wait', 'lol']]